In [9]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 13: Shuttle Search ---</h2><p>Your ferry can make it safely to a nearby port, but it won't get much further. When you call to book another ship, you discover that no ships embark from that port to your vacation island. You'll need to get from the port to the nearest airport.</p>
<p>Fortunately, a shuttle bus service is available to bring you from the sea port to the airport!  Each bus has an ID number that also indicates <em>how often the bus leaves for the airport</em>.</p>
<p>Bus schedules are defined based on a <em>timestamp</em> that measures the <em>number of minutes</em> since some fixed reference point in the past. At timestamp <code>0</code>, every bus simultaneously departed from the sea port. After that, each bus travels to the airport, then various other locations, and finally returns to the sea port to repeat its journey forever.</p>
<p>The time this loop takes a particular bus is also its ID number: the bus with ID <code>5</code> departs from the sea port at timestamps <code>0</code>, <code>5</code>, <code>10</code>, <code>15</code>, and so on. The bus with ID <code>11</code> departs at <code>0</code>, <code>11</code>, <code>22</code>, <code>33</code>, and so on. If you are there when the bus departs, you can ride that bus to the airport!</p>
<p>Your notes (your puzzle input) consist of two lines.  The first line is your estimate of the <em>earliest timestamp you could depart on a bus</em>. The second line lists the bus IDs that are in service according to the shuttle company; entries that show <code>x</code> must be out of service, so you decide to ignore them.</p>
<p>To save time once you arrive, your goal is to figure out <em>the earliest bus you can take to the airport</em>. (There will be exactly one such bus.)</p>
<p>For example, suppose you have the following notes:</p>
<pre><code>939
7,13,x,x,59,x,31,19
</code></pre>
<p>Here, the earliest timestamp you could depart is <code>939</code>, and the bus IDs in service are <code>7</code>, <code>13</code>, <code>59</code>, <code>31</code>, and <code>19</code>. Near timestamp <code>939</code>, these bus IDs depart at the times marked <code>D</code>:</p>
<pre><code>time   bus 7   bus 13  bus 59  bus 31  bus 19
929      .       .       .       .       .
930      .       .       .       D       .
931      D       .       .       .       D
932      .       .       .       .       .
933      .       .       .       .       .
934      .       .       .       .       .
935      .       .       .       .       .
936      .       D       .       .       .
937      .       .       .       .       .
938      D       .       .       .       .
<em>939      .       .       .       .       .</em>
940      .       .       .       .       .
941      .       .       .       .       .
942      .       .       .       .       .
943      .       .       .       .       .
<em>944      .       .       D       .       .</em>
945      D       .       .       .       .
946      .       .       .       .       .
947      .       .       .       .       .
948      .       .       .       .       .
949      .       D       .       .       .
</code></pre>
<p>The earliest bus you could take is bus ID <code>59</code>. It doesn't depart until timestamp <code>944</code>, so you would need to wait <code>944 - 939 = 5</code> minutes before it departs. Multiplying the bus ID by the number of minutes you'd need to wait gives <em><code>295</code></em>.</p>
<p><em>What is the ID of the earliest bus you can take to the airport multiplied by the number of minutes you'll need to wait for that bus?</em></p>
</article>


In [10]:
from math import prod


example = """
939
7,13,x,x,59,x,31,19
"""


def earliest_bus(s: str) -> int:
    timestamp, busses = s.strip().splitlines()
    timestamp = int(timestamp)
    busses = (int(c) for c in busses.split(",") if c.isdecimal())

    return prod(
        min(
            ((b, b * ((timestamp // b) + 1) - timestamp) for b in busses),
            key=lambda t: t[1],
        )
    )


assert earliest_bus(example) == 295

In [11]:
with open("../input/day13.txt") as f:
    puzzle = f.read()

print(f"Part I: {earliest_bus(puzzle)}")

Part I: 261


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>261</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The shuttle company is running a <span title="This is why you should never let me design a contest for a shuttle company.">contest</span>: one gold coin for anyone that can find the earliest timestamp such that the first bus ID departs at that time and each subsequent listed bus ID departs at that subsequent minute. (The first line in your input is no longer relevant.)</p>
<p>For example, suppose you have the same list of bus IDs as above:</p>
<pre><code>7,13,x,x,59,x,31,19</code></pre>
<p>An <code>x</code> in the schedule means there are no constraints on what bus IDs must depart at that time.</p>
<p>This means you are looking for the earliest timestamp (called <code>t</code>) such that:</p>
<ul>
<li>Bus ID <code>7</code> departs at timestamp <code>t</code>.
</li><li>Bus ID <code>13</code> departs one minute after timestamp <code>t</code>.</li>
<li>There are no requirements or restrictions on departures at two or three minutes after timestamp <code>t</code>.</li>
<li>Bus ID <code>59</code> departs four minutes after timestamp <code>t</code>.</li>
<li>There are no requirements or restrictions on departures at five minutes after timestamp <code>t</code>.</li>
<li>Bus ID <code>31</code> departs six minutes after timestamp <code>t</code>.</li>
<li>Bus ID <code>19</code> departs seven minutes after timestamp <code>t</code>.</li>
</ul>
<p>The only bus departures that matter are the listed bus IDs at their specific offsets from <code>t</code>. Those bus IDs can depart at other times, and other bus IDs can depart at those times.  For example, in the list above, because bus ID <code>19</code> must depart seven minutes after the timestamp at which bus ID <code>7</code> departs, bus ID <code>7</code> will always <em>also</em> be departing with bus ID <code>19</code> at seven minutes after timestamp <code>t</code>.</p>
<p>In this example, the earliest timestamp at which this occurs is <em><code>1068781</code></em>:</p>
<pre><code>time     bus 7   bus 13  bus 59  bus 31  bus 19
1068773    .       .       .       .       .
1068774    D       .       .       .       .
1068775    .       .       .       .       .
1068776    .       .       .       .       .
1068777    .       .       .       .       .
1068778    .       .       .       .       .
1068779    .       .       .       .       .
1068780    .       .       .       .       .
<em>1068781</em>    <em>D</em>       .       .       .       .
<em>1068782</em>    .       <em>D</em>       .       .       .
<em>1068783</em>    .       .       .       .       .
<em>1068784</em>    .       .       .       .       .
<em>1068785</em>    .       .       <em>D</em>       .       .
<em>1068786</em>    .       .       .       .       .
<em>1068787</em>    .       .       .       <em>D</em>       .
<em>1068788</em>    D       .       .       .       <em>D</em>
1068789    .       .       .       .       .
1068790    .       .       .       .       .
1068791    .       .       .       .       .
1068792    .       .       .       .       .
1068793    .       .       .       .       .
1068794    .       .       .       .       .
1068795    D       D       .       .       .
1068796    .       .       .       .       .
1068797    .       .       .       .       .
</code></pre>
<p>In the above example, bus ID <code>7</code> departs at timestamp <code>1068788</code> (seven minutes after <code>t</code>). This is fine; the only requirement on that minute is that bus ID <code>19</code> departs then, and it does.</p>
<p>Here are some other examples:</p>
<ul>
<li>The earliest timestamp that matches the list <code>17,x,13,19</code> is <em><code>3417</code></em>.</li>
<li><code>67,7,59,61</code> first occurs at timestamp <em><code>754018</code></em>.</li>
<li><code>67,x,7,59,61</code> first occurs at timestamp <em><code>779210</code></em>.</li>
<li><code>67,7,x,59,61</code> first occurs at timestamp <em><code>1261476</code></em>.</li>
<li><code>1789,37,47,1889</code> first occurs at timestamp <em><code>1202161486</code></em>.</li>
</ul>
<p>However, with so many bus IDs in your list, surely the actual earliest timestamp will be larger than <code>100000000000000</code>!</p>
<p><em>What is the earliest timestamp such that all of the listed bus IDs depart at offsets matching their positions in the list?</em></p>
</article>

</main>


In [58]:
from math import lcm


def earliest_timestamp_bf(s: str) -> int:
    _, busses = s.strip().splitlines()
    busses_delta = [
        (int(b), i) for i, b in enumerate(busses.split(",")) if b.isdecimal()
    ]

    b_max, t_b_max = max(busses_delta)

    t = 0
    while True:
        t += b_max
        t1 = t - t_b_max
        b, tb = busses_delta[0]
        i = 1

        while i < len(busses_delta) and (t1 + tb) % b == 0:
            b, tb = busses_delta[i]
            i += 1

        if (t1 + tb) % b == 0:
            return t1


def chinese_remainder(n, a):
    sum = 0
    product = prod(n)
    for n_i, a_i in zip(n, a):
        p = product // n_i
        sum += a_i * mul_inv(p, n_i) * p
    return sum % product


def mul_inv(a, b):
    b0 = b
    x0, x1 = 0, 1
    if b == 1:
        return 1
    while a > 1:
        q = a // b
        a, b = b, a % b
        x0, x1 = x1 - q * x0, x0
    if x1 < 0:
        x1 += b0
    return x1


def earliest_timestamp(s: str) -> int:
    _, s = s.strip().splitlines()
    n = [int(b) for b in s.split(",") if b.isdecimal()]
    a = [i + int(b) for i, b in enumerate(s.split(",")) if b.isdecimal()]
    return lcm(*n) - chinese_remainder(n, a)


assert earliest_timestamp_bf(example) == 1068781
assert earliest_timestamp_bf("v\n17,x,13,19") == 3417
assert earliest_timestamp_bf("v\n67,7,59,61") == 754018
assert earliest_timestamp_bf("v\n67,x,7,59,61") == 779210
assert earliest_timestamp_bf("v\n67,7,x,59,61") == 1261476
assert earliest_timestamp_bf("v\n1789,37,47,1889") == 1202161486

assert earliest_timestamp(example) == 1068781
assert earliest_timestamp("v\n17,x,13,19") == 3417
assert earliest_timestamp("v\n67,7,59,61") == 754018
assert earliest_timestamp("v\n67,x,7,59,61") == 779210
assert earliest_timestamp("v\n67,7,x,59,61") == 1261476
assert earliest_timestamp("v\n1789,37,47,1889") == 1202161486

In [61]:
print(f"Part II: {earliest_timestamp(puzzle)}")

print("\n\n\n")
print("Input for wolframalpha")
_, n = puzzle.strip().splitlines()
bd = [(int(b), i) for i, b in enumerate(n.split(",")) if b.isdecimal()]

print(", ".join(f"mod[x+{t},{b}]=0" for b, t in bd))

Part II: 807435693182510




Input for wolframalpha
mod[x+0,37]=0, mod[x+27,41]=0, mod[x+37,457]=0, mod[x+50,13]=0, mod[x+51,17]=0, mod[x+60,23]=0, mod[x+66,29]=0, mod[x+68,431]=0, mod[x+87,19]=0


<link href="style.css" rel="stylesheet"></link>

[wolframalpha](https://www.wolframalpha.com/input?i=mod%5Bx%2B0%2C37%5D%3D0%2C+mod%5Bx%2B27%2C41%5D%3D0%2C+mod%5Bx%2B37%2C457%5D%3D0%2C+mod%5Bx%2B50%2C13%5D%3D0%2C+mod%5Bx%2B51%2C17%5D%3D0%2C+mod%5Bx%2B60%2C23%5D%3D0%2C+mod%5Bx%2B66%2C29%5D%3D0%2C++mod%5Bx%2B68%2C431%5D%3D0%2C+mod%5Bx%2B87%2C19%5D%3D0)

Solving:

```json
{
    (x +  0) mod  37 = 0,
    (x + 27) mod  41 = 0,
    (x + 37) mod 457 = 0,
    (x + 50) mod  13 = 0,
    (x + 51) mod  17 = 0,
    (x + 60) mod  23 = 0,
    (x + 66) mod  29 = 0,
    (x + 68) mod 431 = 0,
    (x + 87) mod  19 = 0
} => x = 836856048822287 * n + 807435693182510, n element Z
```

n = 0 gives first solution: 807435693182510

<main>

<p>Your puzzle answer was <code>807435693182510</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>


In [64]:
from math import lcm, prod

_, busses_str = puzzle.strip().splitlines()
n = [int(b) for b in busses_str.split(",") if b.isdecimal()]
ts = [i + int(b) for i, b in enumerate(busses_str.split(",")) if b.isdecimal()]
print(ts)


def chinese_remainder(n, a):
    sum = 0
    product = prod(n)
    for n_i, a_i in zip(n, a):
        p = product // n_i
        sum += a_i * mul_inv(p, n_i) * p
    return sum % product


def mul_inv(a, b):
    b0 = b
    x0, x1 = 0, 1
    if b == 1:
        return 1
    while a > 1:
        q = a // b
        a, b = b, a % b
        x0, x1 = x1 - q * x0, x0
    if x1 < 0:
        x1 += b0
    return x1


print(lcm(*n), chinese_remainder(n, ts), 836856048822287 - 807435693182510)
# So solution is difference between least common multiple and the chinese remainder
assert lcm(*n) - chinese_remainder(n, ts) == 807435693182510
#

[37, 68, 494, 63, 68, 83, 95, 499, 106]
836856048822287 29420355639777 29420355639777


True